# 3. Feature Extraction, Transformations and Cleansing: 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import seaborn as sns
%matplotlib inline
matplotlib.style.use("ggplot")

In [2]:
pd.set_option('display.max_columns', None)

## 3.1. Data Quality: nans, wrong data, etc.

### File loading and examine:

### 1. Vamos a utilizar los data sets desde el 2000 hasta el 2014. Creamos un algoritmo que sea capaz de cargar y concatenar todos los archivos a la vez, previo a  empezar la limpieza

In [3]:
import glob

In [4]:
path =r'../tennis_atp-master/Prueba' # use your path
allFiles = glob.glob(path + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
frame = pd.concat(list_)

### 2. Ahora aplico el proceso de limpieza que hemos dictaminado como necesario:

In [5]:
# 1. Take out matchs from Davies Cup:
framesD = frame[frame['tourney_level'] != 'D']
# 2. Take out matchs from Challenge CUP:
framesDC = framesD[framesD['tourney_level'] != 'C']
# 3.1. Take out marchs'W/O' (Walk Over):
framesDCWO = framesDC[framesDC['score'] != 'W/O']
# 3.2. Take out marchs' W/O' (Walk Over):
framesDCWO = framesDCWO[framesDCWO['score'] != ' W/O']
# 4. Take out matchs with high volume of NaNs:
framesDCWO2 = framesDCWO.dropna(axis=0, thresh=30)

In [6]:
x = frame.shape
y = framesDCWO2.shape
print x
print y

(53120, 49)
(47085, 49)


La limpieza ha supuesto la eliminacion de 5.425 partidos, un 11,5% del total.

### 3. Ahora vamos a analizar variables predictores:
    Cada linea del dataset contiene un partido, cada partido contiene 49 features que describen el torneo, los jugadores y las estadisticas de juego. De cara a modelizar este dataset tenemos que tener en cuenta que hay variables que no van a poder set utilizadas como variables explicativas, bien porque no esten disponibles antes del partido (ej. result, aces, etc.) o bien porque sean variables descriptivas (ej. date, name of the winner, etc.).
    Vamos a utilizar modelos de clasificacion en los que el resultado va a ser 1 gana el jugador A, 0 pierde el jugador A.
    Al tener variables de ambos jugadores podriamos elegir utilizar ambas o hacer una nueva feature con la diferencia entre ellas. Si utilizamos esta segunada visión cualquier partido podría tener un target de 1 o 0 en función de que jugador se ponga primero al hacer un estadístico, por tanto podemos decir que el modelo sería simetrico.
    A continuación vamos a analizar los tipos de variables:
   

#### a) Features constantes:
    Se mantienen constantes en cualquier partido
    1. Hand: Puede ser R o L. Vamos a convertirla en una variable categorica en la que 1 sera R y 0 sera L
    2. Best of: Numero de sets del partido, 3 o 5. Vamos a convertirla en una variable categoriaca en la que 1 sea 3 sets y 0 serán 5 sets.
    3. Height: Puede cambiar en el caso de jugadores muy jovenes, sin embargo, con objeto de simplificar el modelo la mantendremos constante.
    
#### b) Features variables, disponibles en cada partido:
    1. Age: 
    2. Rank/Rank points:
    3. Odds: Esta variable supone una de las claves del proyecto. Originalmente el objetivo de este era construir un modelo de ML que optimizara el ROI de nuestras apuestas. Para optimizar este resultado no solo es necesario tener un indice de acierto importante sino ademas tener en cuenta el retorno de cada acierto o perdida en caso de error. Por tanto para llevar a cabo este objetivo necesitariamos cambiar la funcion de coste de nuestro modelo incluyendo la variable odd. Es decir, la función de coste tiene que maximizar nuestras ganancias. Nuestra funcion de coste seria:
    
    J(w) = 1/2 E
    
    El objetivo de nuestro algoritmo es minimizar la función de coste al maximo. Si el resultado de la función de coste fuese 0 esto supondría que nuestra hipotesis h(x) obtiene la maxima rentabilidad posible.
    
    Dicho de otra manera si en un modelo normal la función de coste para minimizar la distancia suele ser el método de los minimos cuadrados, en este caso habría que ponderar esas distancias con la cantidad a recibir o a perder. 
    
    Para minimizar la función de coste podemos usar diferentes metodos:
    - Algoritmo de descenso del gradiente
    - Algoritmos de optimizacion avanzada
    
    Sin embargo dada la complejidad de este asunto, en este trabajo simplemente la utilizaremos como una variable descriptiva más, siendo el objetivo maximar el % de acierto.
    
#### c)Features variables, no disponibles en cada partido (requieren iteraciones):
    1. Average Minutes por partido
    2. Average Aces por partido
    3. Average Double Faults por partido
    4. % First In (w_1stIn/w_svpt)
    5. % First Won (w_1stWon / w_1stIn)
    6. % Second Won (w_2ndWon / (w_svpt - w_1stIn)) - Crear un estadistico de w_2ndIn
    7. % Break point won ( w_bpSaved / w_bpFaced)

#### d) Features varibles, no disponibles en cada partido (requieren iteraciones muy complejas)
    1. % Victorias en los ultimos 5 partidos
    2. % Victoria Head to Head
    3. % Victorias en los ultimos partidos Head to Head
    4. Surface: No utilizaremos esta variable como tal sino que vamos a poner el % de victorias sobre esa superfice (tendremos que implementar una iteración que busque el % partidos ganados sobre la superficie en que se juegue el partido)
    5. Tourney Level: No utilizaremos esta variable como tal sino que vamos a poner el % de victorias sobre esa superfice (tendremos que implementar una iteración que busque el % partidos ganados sobre la superficie en que se juegue el partido)
    
    Por tanto vamos a disponer de 18 features para poder modelizar.
    Crearemos ademas un variable Retires que utilizaremos para el problema de modelizacion del fraude, pero esto es algo aparte, como un segundo trabajo.

Voy a empezar limpiando el data set de todos aquellos campos que no vamos a utilizar

In [23]:
columns = ['winner_seed', 'loser_seed', 'winner_entry', 'loser_entry']
framesDCWO2.drop(columns, inplace=True, axis=1) 

/Users/NachoSJ/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [25]:
framesDCWO2.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,winner_rank,winner_rank_points,loser_id,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,loser_rank,loser_rank_points,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,Retired,w_hand,l_hand
0,2000-717,Orlando,Clay,32,A,20000501,1,102179,Antony Dupuis,R,185.0,FRA,27.181383,113.0,351.0,102776,Andrew Ilie,R,180.0,AUS,24.035592,50.0,762.0,3-6 7-6(6) 7-6(4),3,R32,162.0,8.0,1.0,126.0,76.0,56.0,29.0,16.0,14.0,15.0,13.0,4.0,110.0,59.0,49.0,31.0,17.0,4.0,4.0,0,1.0,1.0
1,2000-717,Orlando,Clay,32,A,20000501,2,103602,Fernando Gonzalez,R,183.0,CHI,19.756331,352.0,76.0,102821,Cecil Mamiit,R,173.0,PHI,23.843943,139.0,280.0,6-2 7-5,3,R32,86.0,4.0,2.0,67.0,35.0,25.0,16.0,10.0,4.0,6.0,0.0,0.0,57.0,24.0,13.0,17.0,10.0,4.0,9.0,0,1.0,1.0
2,2000-717,Orlando,Clay,32,A,20000501,3,103387,Paradorn Srichaphan,R,185.0,THA,20.881588,103.0,380.0,102205,Sebastien Lareau,R,183.0,CAN,27.011636,133.0,293.0,6-1 6-3,3,R32,64.0,4.0,1.0,46.0,29.0,23.0,11.0,8.0,0.0,0.0,2.0,2.0,65.0,39.0,22.0,10.0,8.0,6.0,10.0,0,1.0,1.0
3,2000-717,Orlando,Clay,32,A,20000501,4,101733,Jan Siemerink,L,183.0,NED,30.047912,107.0,371.0,102925,Justin Gimelstob,R,196.0,USA,23.260780,95.0,408.0,4-6 6-2 7-5,3,R32,150.0,8.0,6.0,109.0,56.0,43.0,21.0,15.0,9.0,12.0,4.0,6.0,104.0,57.0,35.0,24.0,15.0,6.0,11.0,0,0.0,1.0
4,2000-717,Orlando,Clay,32,A,20000501,5,101727,Jason Stoltenberg,R,185.0,AUS,30.075291,74.0,543.0,101826,Alex Lopez Moron,R,175.0,ESP,29.423682,111.0,357.0,6-1 6-4,3,R32,60.0,3.0,0.0,50.0,27.0,22.0,16.0,9.0,1.0,1.0,0.0,3.0,47.0,28.0,17.0,10.0,8.0,3.0,6.0,0,1.0,1.0


In [27]:
framesDCWO2.shape

(47085, 48)

### 4. New Feature creation: Retired. This feature contemple aquellos partidos que no terminaron

In [7]:
Retired = []
for row in framesDCWO2['score']:
    if 'RET' in row:
        # Append 1
        Retired.append(1)
    # else, if not append 0,
    else:
        Retired.append(0)
# Create a column from the list
framesDCWO2['Retired'] = Retired

/Users/NachoSJ/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
framesDCWO2[framesDCWO2['Retired'] == 1 ].shape

(1498, 50)

#### Clasificados como Retired hay 1.327 partidos, lo que supone un 3,2%. Sabemos que hay algunos más que no han sido clasificados como Retired y que tendremos que buscar mas adelante (partido dura menos sets de los programados)

### 5. New feature creation: Hand (1 - Right hand, 0 - Left Hand)

In [13]:
Hand = {'R': 1,'L': 0}

In [14]:
framesDCWO2["w_hand"] = np.nan
framesDCWO2["l_hand"] = np.nan

/Users/NachoSJ/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/NachoSJ/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [15]:
framesDCWO2['w_hand'] = framesDCWO2['w_hand'].fillna(framesDCWO2['winner_hand'].map(Hand))
framesDCWO2['l_hand'] = framesDCWO2['l_hand'].fillna(framesDCWO2['loser_hand'].map(Hand))

/Users/NachoSJ/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/NachoSJ/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [16]:
framesDCWO2.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,winner_rank,winner_rank_points,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,loser_rank,loser_rank_points,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,Retired,w_hand,l_hand
0,2000-717,Orlando,Clay,32,A,20000501,1,102179,NaN,NaN,Antony Dupuis,R,185.0,FRA,27.181383,113.0,351.0,102776,1.0,NaN,Andrew Ilie,R,180.0,AUS,24.035592,50.0,762.0,3-6 7-6(6) 7-6(4),3,R32,162.0,8.0,1.0,126.0,76.0,56.0,29.0,16.0,14.0,15.0,13.0,4.0,110.0,59.0,49.0,31.0,17.0,4.0,4.0,0,1.0,1.0
1,2000-717,Orlando,Clay,32,A,20000501,2,103602,NaN,Q,Fernando Gonzalez,R,183.0,CHI,19.756331,352.0,76.0,102821,NaN,WC,Cecil Mamiit,R,173.0,PHI,23.843943,139.0,280.0,6-2 7-5,3,R32,86.0,4.0,2.0,67.0,35.0,25.0,16.0,10.0,4.0,6.0,0.0,0.0,57.0,24.0,13.0,17.0,10.0,4.0,9.0,0,1.0,1.0
2,2000-717,Orlando,Clay,32,A,20000501,3,103387,NaN,NaN,Paradorn Srichaphan,R,185.0,THA,20.881588,103.0,380.0,102205,NaN,NaN,Sebastien Lareau,R,183.0,CAN,27.011636,133.0,293.0,6-1 6-3,3,R32,64.0,4.0,1.0,46.0,29.0,23.0,11.0,8.0,0.0,0.0,2.0,2.0,65.0,39.0,22.0,10.0,8.0,6.0,10.0,0,1.0,1.0
3,2000-717,Orlando,Clay,32,A,20000501,4,101733,NaN,NaN,Jan Siemerink,L,183.0,NED,30.047912,107.0,371.0,102925,8.0,NaN,Justin Gimelstob,R,196.0,USA,23.260780,95.0,408.0,4-6 6-2 7-5,3,R32,150.0,8.0,6.0,109.0,56.0,43.0,21.0,15.0,9.0,12.0,4.0,6.0,104.0,57.0,35.0,24.0,15.0,6.0,11.0,0,0.0,1.0
4,2000-717,Orlando,Clay,32,A,20000501,5,101727,4.0,NaN,Jason Stoltenberg,R,185.0,AUS,30.075291,74.0,543.0,101826,NaN,NaN,Alex Lopez Moron,R,175.0,ESP,29.423682,111.0,357.0,6-1 6-4,3,R32,60.0,3.0,0.0,50.0,27.0,22.0,16.0,9.0,1.0,1.0,0.0,3.0,47.0,28.0,17.0,10.0,8.0,3.0,6.0,0,1.0,1.0


 Voy a crear primero todos los pares de features y no haré las diferencias hasta el final. El motivo es que el dataset siempre esta ordenado con winner loser por tanto si aplicara la formula a-b a todo el dataset la Y siempre sería 1, es decir el resultado sería siempre victoria, y el modelo no funcionaria correctamente.
 
 Para solucionar este problema y poder incluir en el modelo partidos con el target 0, voy a aprovechar la simetría de mi modelo. Dividiré el data set en 2 utilizando la lógica siguiente para cada una de las partes:
 
 1 = winner - loser                  
 0 = - winner + loser

De esta forma voy a obtener un data set con resultados en ambos sentidos

### 6. New feature creation: Best of (1 - 3 sets, 0 - 5 sets)

### 7. Solve *_ht (height) problem:

#### Ahora tenemos que hacer un diccionario con todos los *_ht que faltan:

In [17]:
framesDCWO2.isnull().sum()

tourney_id                0
tourney_name              0
surface                   0
draw_size                 0
tourney_level             0
tourney_date              0
match_num                 0
winner_id                 0
winner_seed           25252
winner_entry          40954
winner_name               0
winner_hand               0
winner_ht              1447
winner_ioc                0
winner_age                0
winner_rank             603
winner_rank_points      603
loser_id                  0
loser_seed            35437
loser_entry           36789
loser_name                0
loser_hand                0
loser_ht               2591
loser_ioc                 0
loser_age                 2
loser_rank              688
loser_rank_points       688
score                     0
best_of                   0
round                     0
minutes                1295
w_ace                     0
w_df                      0
w_svpt                    0
w_1stIn                   0
w_1stWon            

In [18]:
w_Names = {'Adrian Cruciat': 185,
 'Alejandro Gonzalez': 191,
 'Aleksandr Nedovyesov': 193,
 'Alexander Zverev': 191,
 'Alexandre Sidorenko': 185,
 'Ante Pavic': 196,
 'Augustin Gensse': 180,
 'Austin Krajicek': 188,
 'Axel Michon': 176,
 'Bastian Knittel': 180,
 'Blaz Rola': 193,
 'Borna Coric': 185,
 'Chris Eaton': 188,
 'Christian Garin': 185,
 'Christian Harrison': 180,
 'Christian Lindell': 193,
 'Chuhan Wang': 188,
 'Daniel Kosakowski': 185,
 'Daniel Munoz De La Nava': 178,
 'Dennis Novikov': 193,
 'Diego Sebastian Schwartzman': 170,
 'Dimitar Kutrovsky': 175,
 'Dominic Thiem': 185,
 'Elias Ymer': 183,
 'Emilio Gomez': 185,
 'Facundo Bagnis': 183,
 'Filip Peliwo': 178,
 'Gianluca Naso': 193,
 'Guido Pella': 185,
 'Guillermo Olaso': 175,
 'Gustavo Marcaccio': 170,
 'Henri Kontinen': 191,
 'Henri Laaksonen': 185,
 'Hiroki Kondo': 178,
 'Hiroki Moriya': 170,
 'Inigo Cervantes Huegun': 183,
 'Ivan Sergeyev': 175,
 'Jamie Baker': 183,
 'Jan Lennard Struff': 196,
 'Jaroslav Pospisil': 178,
 'Jason Kubler': 178,
 'Javier Marti': 185,
 'Jiri Vesely': 198,
 'Joao Sousa': 185,
 'Jonathan Dasnieres De Veigy': 175,
 'Jonathan Marray': 183,
 'Jorge Aguilar': 178,
 'Karen Khachanov': 198,
 'Kyle Edmund': 188,
 'Laurent Recouderc': 178,
 'Leonardo Tavares': 185,
 'Lucas Pouille': 185,
 'Luka Gregorc': 190,
 'Luke Saville': 188,
 'Marco Trungelliti': 180,
 'Martin Alund': 183,
 'Martin Fischer': 180,
 'Marton Fucsovics': 188,
 'Mate Delic': 196,
 'Mate Pavic': 193,
 'Mathieu Rodrigues': 180,
 'Matteo Viola': 185,
 'Maxime Teixeira': 188,
 'Miloslav Mecir': 196,
 'Mirza Basic': 188,
 'Nick Kyrgios': 193,
 'Nikola Milojevic': 188,
 'Nils Langer': 193,
 'Pablo Carreno Busta': 188,
 'Patrik Rosenholm': 178,
 'Pavol Cervenak': 191,
 'Peng Sun': 184,
 'Petar Jelenic': 190,
 'Philip Davydenko': 183,
 'Philipp Oswald': 201,
 'Pierre Hugues Herbert': 188,
 'Pierre Ludovic Duclos': 188,
 'Radu Albot': 175,
 'Rafael Arevalo': 180,
 'Ramkumar Ramanathan': 188,
 'Renzo Olivo': 180,
 'Rhyne Williams': 185,
 'Roberto Bautista Agut': 183,
 'Roberto Carballes Baena': 180,
 'Ruben Bemelmans': 183,
 'Samuel Groth': 193,
 'Sebastian Rieschick': 188,
 'Sergei Bubka': 188,
 'Takahiro Terachi': 174,
 'Taro Daniel': 191,
 'Thanasi Kokkinakis': 196,
 'Tim Puetz': 185,
 'Yan Bai': 185,
 'Yannick Mertens': 188,
 'Yuichi Sugita': 175,
 'Ze Zhang': 188}

In [19]:
framesDCWO2['winner_ht'] = framesDCWO2['winner_ht'].fillna(framesDCWO2['winner_name'].map(w_Names))

/Users/NachoSJ/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [20]:
framesDCWO2.isnull().sum()

tourney_id                0
tourney_name              0
surface                   0
draw_size                 0
tourney_level             0
tourney_date              0
match_num                 0
winner_id                 0
winner_seed           25252
winner_entry          40954
winner_name               0
winner_hand               0
winner_ht               223
winner_ioc                0
winner_age                0
winner_rank             603
winner_rank_points      603
loser_id                  0
loser_seed            35437
loser_entry           36789
loser_name                0
loser_hand                0
loser_ht               2591
loser_ioc                 0
loser_age                 2
loser_rank              688
loser_rank_points       688
score                     0
best_of                   0
round                     0
minutes                1295
w_ace                     0
w_df                      0
w_svpt                    0
w_1stIn                   0
w_1stWon            

### Antes de hacer ninguna variable tenemos que limpiar de Nans y demas, asi com incluir los odds

### 9. Average minutes per match

Para poder hacer cualquier iteracion lo primero que tenemos que hacer es ordenar cronologicamente, ya que la iteración buscará la información en partidos pasados, obviando los partidos no ocurridos antes de esa fecha.
Basicamente la iteración buscará en cada partido para ambos jugadores los 'minutes' de todos aquellos partidos (aparezca como winner o loser) pasados. Para hacer la media simplemente sumará todos los minutes y lo dividirá por el total de partidos.

In [30]:
framesDCWO2 = framesDCWO2.sort_values(by=['tourney_date', 'match_num'], ascending=False)

In [33]:
framesDCWO2.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,winner_rank,winner_rank_points,loser_id,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,loser_rank,loser_rank_points,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,Retired,w_hand,l_hand
2675,2016-0605,London,Hard,8,A,20161114,300,104918,Andy Murray,R,190.0,GBR,29.503080,1.0,11185.0,104925,Novak Djokovic,R,188.0,SRB,29.483915,2.0,10780.0,6-3 6-4,3,F,103.0,3.0,3.0,59.0,32.0,27.0,13.0,10.0,0.0,1.0,1.0,0.0,61.0,44.0,26.0,11.0,9.0,6.0,9.0,0,1.0,1.0
2676,2016-0605,London,Hard,8,A,20161114,299,104918,Andy Murray,R,190.0,GBR,29.503080,1.0,11185.0,105683,Milos Raonic,R,196.0,CAN,25.883641,4.0,5050.0,5-7 7-6(5) 7-6(9),3,SF,218.0,7.0,6.0,146.0,89.0,55.0,36.0,18.0,8.0,12.0,10.0,6.0,128.0,81.0,60.0,21.0,18.0,4.0,7.0,0,1.0,1.0
2677,2016-0605,London,Hard,8,A,20161114,298,104925,Novak Djokovic,R,188.0,SRB,29.483915,2.0,10780.0,105453,Kei Nishikori,R,178.0,JPN,26.877481,5.0,4705.0,6-1 6-1,3,SF,66.0,3.0,1.0,44.0,28.0,23.0,7.0,7.0,2.0,3.0,4.0,1.0,55.0,35.0,18.0,4.0,7.0,5.0,11.0,0,1.0,1.0
2678,2016-0605,London,Hard,8,A,20161114,297,104918,Andy Murray,R,190.0,GBR,29.503080,1.0,11185.0,104527,Stanislas Wawrinka,R,183.0,SUI,31.633128,3.0,5115.0,6-4 6-2,3,RR,86.0,3.0,2.0,51.0,30.0,26.0,12.0,9.0,2.0,2.0,7.0,1.0,67.0,41.0,27.0,11.0,9.0,4.0,7.0,0,1.0,1.0
2679,2016-0605,London,Hard,8,A,20161114,296,104918,Andy Murray,R,190.0,GBR,29.503080,1.0,11185.0,105453,Kei Nishikori,R,178.0,JPN,26.877481,5.0,4705.0,6-7(9) 6-4 6-4,3,RR,200.0,8.0,4.0,132.0,70.0,53.0,30.0,16.0,9.0,11.0,0.0,6.0,119.0,75.0,54.0,18.0,16.0,3.0,7.0,0,1.0,1.0


Ahora ya puedo iterar

Ok! He conseguido hacer la suma acumulativa. Ahora tengo que condicionar la suma a que sea el mismo jugador. Por tanto tendre que hacer un 'w_minutes_cum' y un 'l_minutes_cum'

In [55]:
# framesDCWO2.groupby(by=['winner_name','loser_name']).count().groupby(level=[0]).cumsum()

tourney_id  tourney_name  surface  \
winner_name      loser_name                                                   
Adam Chadaj      Alessio Di Mauro                  1             1        1   
Adam Pavlasek    Marcel Granollers                 1             1        1   
                 Maximo Gonzalez                   2             2        2   
                 Roberto Carballes Baena           3             3        3   
Adrian Cruciat   Filippo Volandri                  1             1        1   
Adrian Garcia    Galo Blanco                       1             1        1   
                 Gregory Carraz                    2             2        2   
                 James Blake                       3             3        3   
                 Kim Tiilikainen                   4             4        4   
                 Marc Rosset                       5             5        5   
                 Nikolay Davydenko                 6             6        6   
                 Olivier Mutis                     7             7        7   
                 Paul Capdeville                   8             8        8   
                 Satoshi Iwabuchi                  9             9        9   
Adrian Mannarino Albert Ramos                      2             2        2   
                 Alejandro Falla                   3             3        3   
                 Alejandro Gonzalez                4             4        4   
                 Alexandr Dolgopolov               5             5        5   
                 Aljaz Bedene                      6             6        6   
                 Andreas Seppi                     8             8        8   
                 Arnaud Clement                    9             9        9   
                 Benoit Paire                     11            11       11   
                 Blaz Rola                        12            12       12   
                 Carlos Berlocq                   13            13       13   
                 Conor Niland                     14            14       14   
                 Damir Dzumhur                    15            15       15   
                 Daniel Brands                    16            16       16   
                 Daniel Cox                       17            17       17   
                 Daniil Medvedev                  18            18       18   
                 Denis Gremelmayr                 19            19       19   
...                                              ...           ...      ...   
Yuri Schukin     Mikhail Youzhny                   8             8        8   
                 Rajeev Ram                        9             9        9   
                 Robby Ginepri                    10            10       10   
                 Simone Bolelli                   11            11       11   
                 Werner Eschauer                  12            12       12   
Yves Allegro     Alex Calatrava                    1             1        1   
                 Andrew Ilie                       2             2        2   
                 Antony Dupuis                     3             3        3   
                 Kristian Pless                    4             4        4   
Zack Fleishman   Agustin Calleri                   1             1        1   
                 Fernando Gonzalez                 2             2        2   
                 Joachim Johansson                 3             3        3   
                 Kei Nishikori                     4             4        4   
                 Michael Llodra                    5             5        5   
                 Robert Kendrick                   6             6        6   
                 Vincent Spadea                    7             7        7   
Ze Zhang         Denis Istomin                     1             1        1   
                 Di Wu                             2             2        2   
             

In [53]:
# framesDCWO2.groupby(by=['winner_name','loser_name']).sum().groupby(level=[0]).cumsum()

draw_size  tourney_date  match_num  \
winner_name      loser_name                                                    
Adam Chadaj      Alessio Di Mauro                32      20050801         10   
Adam Pavlasek    Marcel Granollers               32      20160718        288   
                 Maximo Gonzalez                 64      40321436        563   
                 Roberto Carballes Baena        192      60481959        708   
Adrian Cruciat   Filippo Volandri                32      20080908          3   
Adrian Garcia    Galo Blanco                     32      20000228         14   
                 Gregory Carraz                  64      40050331         22   
                 James Blake                     96      60091147         26   
                 Kim Tiilikainen                128      80091864         33   
                 Marc Rosset                    160     100132576         39   
                 Nikolay Davydenko              192     120173385         55   
                 Olivier Mutis                  224     140214194         79   
                 Paul Capdeville                256     160254403         82   
                 Satoshi Iwabuchi               288     180295330         96   
Adrian Mannarino Albert Ramos                   128      40311328        321   
                 Alejandro Falla                156      60461440        343   
                 Alejandro Gonzalez             184      80592155        350   
                 Alexandr Dolgopolov            212     100733168        365   
                 Aljaz Bedene                   260     120883789        385   
                 Andreas Seppi                  324     161125324        676   
                 Arnaud Clement                 352     181235434        695   
                 Benoit Paire                   512     221556257       1261   
                 Blaz Rola                      640     241706376       1271   
                 Carlos Berlocq                 668     261856893       1275   
                 Conor Niland                   796     281967513       1322   
                 Damir Dzumhur                  844     302108330       1338   
                 Daniel Brands                  872     322239253       1347   
                 Daniel Cox                     928     342379862       1370   
                 Daniil Medvedev                960     362540468       1660   
                 Denis Gremelmayr               992     382650599       1678   
...                                             ...           ...        ...   
Yuri Schukin     Mikhail Youzhny                268     160625930         69   
                 Rajeev Ram                     396     180726454        115   
                 Robby Ginepri                  452     200806882        132   
                 Simone Bolelli                 484     220877792        149   
                 Werner Eschauer                540     240948215        173   
Yves Allegro     Alex Calatrava                  32      20020617         10   
                 Andrew Ilie                     64      40031235         21   
                 Antony Dupuis                   96      60051852         42   
                 Kristian Pless                 128      80122469         47   
Zack Fleishman   Agustin Calleri                128      20070115         20   
                 Fernando Gonzalez              160      40140831         21   
                 Joachim Johansson              192      60171538         24   
                 Kei Nishikori                  240      80242539         36   
                 Michael Llodra                 272     100272749         51   
                 Robert Kendrick                304     120343465         68   
                 Vincent Spadea                 336     140394190         77   
Ze Zhang         Denis Istomin                   32      20151005          2   
                 Di Wu                    

Estas no sirven para esto pero me pueden servir para el head to head

In [68]:
framesDCWO2 = framesDCWO2.set_index('tourney_date','match_num').sort_index(ascending=False)

In [76]:
framesDCWO2['w_minutes_cum']=framesDCWO2.groupby('winner_name')['minutes'].transform(pd.Series.cumsum)

In [73]:
framesDCWO2.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,match_num,winner_id,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,winner_rank,winner_rank_points,loser_id,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,loser_rank,loser_rank_points,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,Retired,w_hand,l_hand,minutes_cum,w_minutes_cum
tourney_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20161114,2016-0605,London,Hard,8,A,300,104918,Andy Murray,R,190.0,GBR,29.503080,1.0,11185.0,104925,Novak Djokovic,R,188.0,SRB,29.483915,2.0,10780.0,6-3 6-4,3,F,103.0,3.0,3.0,59.0,32.0,27.0,13.0,10.0,0.0,1.0,1.0,0.0,61.0,44.0,26.0,11.0,9.0,6.0,9.0,0,1.0,1.0,103.0,103.0
20161114,2016-0605,London,Hard,8,A,299,104918,Andy Murray,R,190.0,GBR,29.503080,1.0,11185.0,105683,Milos Raonic,R,196.0,CAN,25.883641,4.0,5050.0,5-7 7-6(5) 7-6(9),3,SF,218.0,7.0,6.0,146.0,89.0,55.0,36.0,18.0,8.0,12.0,10.0,6.0,128.0,81.0,60.0,21.0,18.0,4.0,7.0,0,1.0,1.0,321.0,321.0
20161114,2016-0605,London,Hard,8,A,298,104925,Novak Djokovic,R,188.0,SRB,29.483915,2.0,10780.0,105453,Kei Nishikori,R,178.0,JPN,26.877481,5.0,4705.0,6-1 6-1,3,SF,66.0,3.0,1.0,44.0,28.0,23.0,7.0,7.0,2.0,3.0,4.0,1.0,55.0,35.0,18.0,4.0,7.0,5.0,11.0,0,1.0,1.0,387.0,66.0
20161114,2016-0605,London,Hard,8,A,297,104918,Andy Murray,R,190.0,GBR,29.503080,1.0,11185.0,104527,Stanislas Wawrinka,R,183.0,SUI,31.633128,3.0,5115.0,6-4 6-2,3,RR,86.0,3.0,2.0,51.0,30.0,26.0,12.0,9.0,2.0,2.0,7.0,1.0,67.0,41.0,27.0,11.0,9.0,4.0,7.0,0,1.0,1.0,473.0,407.0
20161114,2016-0605,London,Hard,8,A,296,104918,Andy Murray,R,190.0,GBR,29.503080,1.0,11185.0,105453,Kei Nishikori,R,178.0,JPN,26.877481,5.0,4705.0,6-7(9) 6-4 6-4,3,RR,200.0,8.0,4.0,132.0,70.0,53.0,30.0,16.0,9.0,11.0,0.0,6.0,119.0,75.0,54.0,18.0,16.0,3.0,7.0,0,1.0,1.0,673.0,607.0


## YES! Esta es la forma de hacerlo, el problema es que esta ordenado alreves

Ahora hay que crear otra columna que lleva la cuenta del numero de partidos, para poder hacer la media:

In [79]:
framesDCWO2['Count'] = 1

In [81]:
framesDCWO2['w_minutes_count']=framesDCWO2.groupby('winner_name')['Count'].transform(pd.Series.cumsum)

In [82]:
framesDCWO2.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,match_num,winner_id,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,winner_rank,winner_rank_points,loser_id,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,loser_rank,loser_rank_points,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,Retired,w_hand,l_hand,minutes_cum,w_minutes_cum,w_minutes_count,Count
tourney_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20161114,2016-0605,London,Hard,8,A,300,104918,Andy Murray,R,190.0,GBR,29.503080,1.0,11185.0,104925,Novak Djokovic,R,188.0,SRB,29.483915,2.0,10780.0,6-3 6-4,3,F,103.0,3.0,3.0,59.0,32.0,27.0,13.0,10.0,0.0,1.0,1.0,0.0,61.0,44.0,26.0,11.0,9.0,6.0,9.0,0,1.0,1.0,103.0,103.0,1,1
20161114,2016-0605,London,Hard,8,A,299,104918,Andy Murray,R,190.0,GBR,29.503080,1.0,11185.0,105683,Milos Raonic,R,196.0,CAN,25.883641,4.0,5050.0,5-7 7-6(5) 7-6(9),3,SF,218.0,7.0,6.0,146.0,89.0,55.0,36.0,18.0,8.0,12.0,10.0,6.0,128.0,81.0,60.0,21.0,18.0,4.0,7.0,0,1.0,1.0,321.0,321.0,2,1
20161114,2016-0605,London,Hard,8,A,298,104925,Novak Djokovic,R,188.0,SRB,29.483915,2.0,10780.0,105453,Kei Nishikori,R,178.0,JPN,26.877481,5.0,4705.0,6-1 6-1,3,SF,66.0,3.0,1.0,44.0,28.0,23.0,7.0,7.0,2.0,3.0,4.0,1.0,55.0,35.0,18.0,4.0,7.0,5.0,11.0,0,1.0,1.0,387.0,66.0,1,1
20161114,2016-0605,London,Hard,8,A,297,104918,Andy Murray,R,190.0,GBR,29.503080,1.0,11185.0,104527,Stanislas Wawrinka,R,183.0,SUI,31.633128,3.0,5115.0,6-4 6-2,3,RR,86.0,3.0,2.0,51.0,30.0,26.0,12.0,9.0,2.0,2.0,7.0,1.0,67.0,41.0,27.0,11.0,9.0,4.0,7.0,0,1.0,1.0,473.0,407.0,3,1
20161114,2016-0605,London,Hard,8,A,296,104918,Andy Murray,R,190.0,GBR,29.503080,1.0,11185.0,105453,Kei Nishikori,R,178.0,JPN,26.877481,5.0,4705.0,6-7(9) 6-4 6-4,3,RR,200.0,8.0,4.0,132.0,70.0,53.0,30.0,16.0,9.0,11.0,0.0,6.0,119.0,75.0,54.0,18.0,16.0,3.0,7.0,0,1.0,1.0,673.0,607.0,4,1


#### OK. Ya tengo la cuenta. Ahora deberia ordenar correctamente el df y y obtener el resultado de:
    w_minutes_cum
    w_minutes_count
    w_minutes_average
    

## 3.2. Data Transformation:

#### 1. Creating new Features: Sets

In [ ]:
# Separo los sets:
framesDCWO2['s1'], framesDCWO2['s5'] = framesDCWO2['score'].str.split(' ', 1).str
framesDCWO2['s2'], framesDCWO2['s5'] = framesDCWO2['s5'].str.split(' ', 1).str
framesDCWO2['s3'], framesDCWO2['s5'] = framesDCWO2['s5'].str.split(' ', 1).str
framesDCWO2['s4'], framesDCWO2['s5'] = framesDCWO2['s5'].str.split(' ', 1).str
# Separo los juegos:
framesDCWO2['w_s1'], framesDCWO2['l_s1'] = framesDCWO2['s1'].str.split('-', 1).str
framesDCWO2['w_s2'], framesDCWO2['l_s2'] = framesDCWO2['s2'].str.split('-', 1).str
framesDCWO2['w_s3'], framesDCWO2['l_s3'] = framesDCWO2['s3'].str.split('-', 1).str
framesDCWO2['w_s4'], framesDCWO2['l_s4'] = framesDCWO2['s4'].str.split('-', 1).str
framesDCWO2['w_s5'], framesDCWO2['l_s5'] = framesDCWO2['s5'].str.split('-', 1).str
# Quito los tiebreaks

#### 2. Eliminating Features:

In [ ]:
el framesDCWO2['s1']
del framesDCWO2['s2']
del framesDCWO2['s3']
del framesDCWO2['s4']
del framesDCWO2['s5']
# Cuando repase todo hacer reengeniere:
# df.drop(df.columns[[0, 1, 3]], axis=1)

In [24]:
framesDCWO2.columns

Index([u'tourney_id', u'tourney_name', u'surface', u'draw_size',
       u'tourney_level', u'tourney_date', u'match_num', u'winner_id',
       u'winner_seed', u'winner_entry', u'winner_name', u'winner_hand',
       u'winner_ht', u'winner_ioc', u'winner_age', u'winner_rank',
       u'winner_rank_points', u'loser_id', u'loser_seed', u'loser_entry',
       u'loser_name', u'loser_hand', u'loser_ht', u'loser_ioc', u'loser_age',
       u'loser_rank', u'loser_rank_points', u'score', u'best_of', u'round',
       u'minutes', u'w_ace', u'w_df', u'w_svpt', u'w_1stIn', u'w_1stWon',
       u'w_2ndWon', u'w_SvGms', u'w_bpSaved', u'w_bpFaced', u'l_ace', u'l_df',
       u'l_svpt', u'l_1stIn', u'l_1stWon', u'l_2ndWon', u'l_SvGms',
       u'l_bpSaved', u'l_bpFaced', u'Retired'],
      dtype='object')

## 3.3. Data Analisis:

### 3.3.1. Feature distribution: